<a href="https://colab.research.google.com/github/nupursjsu/Deep-Learning/blob/master/Ungraded_Assignment5/Keras_save_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstrating saving and loading of Keras models

## Importing necessary libraries

In [1]:
!pip install pyyaml h5py
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.2.0


## Loading MNIST dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Building the model

In [3]:
# Define a simple sequential model
def build_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = build_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Saving checkpoints during training

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 1.2316 - accuracy: 0.6375
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 1.2057 - accuracy: 0.6460 - val_loss: 0.7353 - val_accuracy: 0.7940
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4382 - accuracy: 0.8761
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4393 - accuracy: 0.8770 - val_loss: 0.5636 - val_accuracy: 0.8200
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2964 - accuracy: 0.9208
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2944 - accuracy: 0.9210 - val_loss: 0.4997 - val_accuracy: 0.8390
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.2149 - accuracy: 0.9510
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [5]:
#createing a single collection of TensorFlow checkpoint files
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [7]:
#Creating model instance
model = build_model()

#Evaluating the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3376 - accuracy: 0.1100
Untrained model, accuracy: 11.00%


In [8]:
# Loading the weights from the checkpoint and re-evaluating the model
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4022 - accuracy: 0.8580
Restored model, accuracy: 85.80%


## Saving weights of model manually

In [10]:
model.save_weights('./checkpoints/my_checkpoint')

#Creating model instance and restoring weights
model = build_model()
model.load_weights('./checkpoints/my_checkpoint')

#Evaluating the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4022 - accuracy: 0.8580
Restored model, accuracy: 85.80%


## Saving the entire model in SavedModel format

In [11]:
#Creating a new model instance and saving the entire model
model = build_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1875 - accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4314 - accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2975 - accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2243 - accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1593 - accuracy: 0.9660
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [12]:
#Reloading the save model
new_model = tf.keras.models.load_model('saved_model/my_model')

#Checking the architecture of the reloaded model
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


It can be seen it is the same as the original model.

## Saving the entire model in HDF5 format

In [14]:
#Creating a new model instance and saving the entire model as HDF5 file
model = build_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1698 - accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4109 - accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2822 - accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2121 - accuracy: 0.9430
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1500 - accuracy: 0.9670


In [15]:
#Reloading the save model
new_model = tf.keras.models.load_model('my_model.h5')

#Checking the architecture of the reloaded model
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
#Checking its accuracy
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4370 - accuracy: 0.8570
Restored model, accuracy: 85.70%


## Conclusion

Using Keras save and load you can save the entire models with the same architecture, weight parameters etc